In [118]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/BRACBANK.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP   TRADE  VALUE (mn)     VOLUME
0  439 2018-05-08     BRACBANK  68.3  ...  69.1   588.0      34.442   502038.0
1  438 2018-06-08     BRACBANK  71.6  ...  68.2  1826.0     125.978  1796391.0
2  437 2018-07-08     BRACBANK  73.7  ...  71.3  1811.0     111.683  1525306.0
3  436 2018-08-08     BRACBANK  74.3  ...  73.6  1900.0     144.013  1952596.0
4  435 2018-09-08     BRACBANK  73.5  ...  74.5  1207.0      83.986  1136820.0

[5 rows x 12 columns]

In [119]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE  LTP*  ...   YCP  TRADE  VALUE (mn)     VOLUME
36  403 2018-01-10     BRACBANK  71.5  ...  71.8  308.0      10.398   144998.0
59  380 2018-01-11     BRACBANK  69.6  ...  70.4  703.0      46.859   667978.0
37  402 2018-02-10     BRACBANK  72.3  ...  71.3  352.0      12.633   175816.0
79  360 2018-02-12     BRACBANK  72.9  ...  72.7  247.0      15.192   209760.0
16  423 2018-03-09     BRACBANK  76.7  ...  77.0  952.0      84.725  1104301.0

[5 rows x 12 columns]

In [120]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE  LTP*  HIGH  ...   YCP  TRADE  VALUE (mn)    VOLUME
DATE                                      ...                                   
2018-01-10  403     BRACBANK  71.5  72.4  ...  71.8  308.0      10.398  144998.0
2018-01-11  380     BRACBANK  69.6  70.6  ...  70.4  703.0      46.859  667978.0
2018-02-10  402     BRACBANK  72.3  72.4  ...  71.3  352.0      12.633  175816.0

[3 rows x 11 columns]

In [121]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [122]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [123]:
df['volatility'] = df['volatility']*100

In [124]:
import matplotlib.pyplot as plt 

In [125]:
df=df.reset_index()['volatility']

In [126]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [127]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [128]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [129]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [130]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression



In [131]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [132]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [133]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [134]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [135]:
model4.fit(X_train, y_train)

[16:35:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [136]:
train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [137]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [138]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [139]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [140]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [141]:
Y_test =scaler.inverse_transform(Y_test)

In [142]:
Y_test

array([[17.15806749, 13.36726854, 13.36726854, 11.24067114, 11.24067114,
        11.24067114, 11.24067114, 11.24067114, 11.24067114, 11.24067114,
        11.24067114, 11.24067114, 11.24067114, 14.94976376, 15.84083871,
        16.15588539, 16.15588539, 16.7584975 , 15.24011493, 16.73421407,
        16.78685133, 16.78685133, 19.05954192, 19.75470198, 19.9824604 ,
        22.31636981, 23.13042591, 23.31064053, 25.36381592, 25.3927773 ,
        26.23070224]])

In [143]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))

In [144]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [145]:
test_predict1

array([[12.89803462, 12.89732109, 12.8355682 , 12.83132755, 10.22224629,
        10.08171325, 10.08475583, 10.03260441, 10.10979141, 10.06988917,
        10.07272004, 10.12266849, 10.17051007, 10.21277308, 12.74449126,
        12.88773677, 12.88962651, 12.89636264, 12.90359349, 12.90949535,
        12.90772309, 12.90854809, 12.90812368, 12.90855864, 12.94431871,
        12.94792995, 12.95062639, 12.95139877, 12.95852087, 12.95262332,
        12.95842382]])

In [146]:
test_predict2

array([[12.1929205 , 12.17071903, 12.15185299, 12.16050577, 10.33160422,
        10.3224433 , 10.3224433 , 10.30559521, 10.31525385, 10.31116164,
        10.31525385, 10.31627764, 10.31927553, 10.32533072, 11.44436221,
        12.08585682, 12.10784703, 12.18970476, 12.18970476, 12.18970476,
        12.18859123, 12.18970476, 12.19837809, 12.19837809, 12.2691673 ,
        12.30165788, 12.2864751 , 12.2864751 , 12.32274349, 12.33046376,
        12.33046376]])

In [147]:
test_predict3

array([[10.95704201, 11.01662998, 11.00649853, 10.78864785, 10.45798603,
        10.16140063, 10.11310941,  9.39367553,  9.43868444,  9.58499943,
         9.65377996,  9.48454635,  9.28081897,  9.28224587, 10.69829974,
        10.7060144 , 10.9306515 , 10.92462317, 10.93883065, 10.88918705,
        10.82043661, 11.08792101, 10.73274127, 11.43326053, 11.68779043,
        11.20847691, 11.22588249, 11.21533995, 11.21445941, 11.20962669,
        11.40967351]])

In [148]:
test_predict4

array([[11.677043 , 11.854102 , 11.631462 , 11.625792 , 10.447625 ,
        10.394013 , 10.394013 ,  9.646739 ,  9.704585 ,  9.784447 ,
         9.78726  ,  9.555973 ,  9.555973 ,  9.663596 , 10.911959 ,
        10.911959 , 10.911959 , 10.911959 , 10.911959 , 10.911959 ,
        10.922328 , 10.642714 , 10.642714 , 11.093924 , 11.702528 ,
        11.159177 , 11.164846 , 11.164846 , 11.3555565, 11.789011 ,
        11.789011 ]], dtype=float32)

In [149]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

5.872274954411394

In [150]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

6.324165249875114

In [151]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

7.1432482301530245

In [152]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

7.043643686934979

In [153]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 4.427772


In [154]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 4.810291


In [155]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 5.719238


In [156]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 5.561254
